# WiFi Localisation Starting Notebook
## Cyberthon 2022 organised by CSIT, DCO and HCI

This notebook is to help people interact with the data, you may use it if you want to or not.

ALT-TAB LABS LLP &copy; 2019 - 2022 All Rights Reserved

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# feel free to add any library you need

In [2]:
# load training data
df0 = pd.read_json("./wifi_train.json")
df0.describe()

,9vdEmSoDVo5bPvTqLEci,qOlzm9dWsWbSTRqfc9x0,1mlZkF81youDiadsCub0,capZWyIwXHbhuXh67dPx,jo2EGFLxlr5pEtUr2hIw,2HJ59WoZEY8QYMDeVNJ9,cS565f9XQWeyYG9etG9R,dU0La1yLqRDPqAbxI0g9,6qxhQwhZsas7S9HkcGmv,e4SYxYNJiyHRpdWb50uK,...,FMIDAN951McoonAtWDFE,ztPqW3dRWXz60Bw8ubjg,2HVZqpty0FRBGl66Fhqd,McAmTquLrT1q1nbpWuzA,9KIssMatNQMu20uCPkZO,DsxvYPLVDQQdbXCjlGCe,WHJfDSbNggDnH1mRzSoX,et9ILG7uZyJIdBZgLOk8,pSwO7lLaIpNDVPvwEroZ,DboAV4VBoOIrkMdMhmjL
count,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
unique,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
top,"{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...",...,"{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'18:64:72:56:12:f0': [-91], '18:64:72:56:44:5...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'18:64:72:56:2c:88': [-90], '18:64:72:56:44:4...","{'__datatype__': 'timestamp', 'value': {'_seco..."
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [3]:
# preprocess your data

# sample iteration over df0
for ssid in df0:
    locationX = df0[ssid]['location']['x']
    locationY = df0[ssid]['location']['y']
    # print(ssid, locationX, locationY) # this prints 404 lines
    # do some other stuff you want to

In [ ]:
# format training data
# do train_test_split

In [ ]:
# define your model

In [ ]:
# do your training here

In [ ]:
# do your validation here

In [ ]:
# use plt to visualise your model's performance (optional)
#
# fig, ax = plt.subplots(figsize=(10,10))
# ax.scatter(y_true_x, y_true_y) # change these variables to your ones
# ax.scatter(y_pred_x, y_pred_y) # change these variables to your ones

In [4]:
# load test data
df1 = pd.read_json("./wifi_test.json")
df1.describe()

,KnW7bcCTFKSkaxQM9HwS,WVdanpyXB4Le6D7Qlxqm,yr3L2JcYm938dXdtCYj0,9MGDml7ZSWLVrxkytoR8,bFesNkqRjhgvo7bAHQs6,9xtGXaudjqX40DDTw2wc,x73WexagJ3epot4Vt5hI,HFne5SeGBk44BKDH4zS2,w8Pjg1FP9CweGmwiNdaM,zcjOWinxMd4Xw3aysC9x,...,dpmzX8WRwIywLxWZLVAd,GQ3QR7GIeS1NZxQnYfHR,zdI1UZe1hKbMzxAEAnoi,ggOpXDbzjI5hLVh2G6cn,GU4HX4y0l1Pm8cGukyKA,dHXIwA3dTzHRZjpxlA3Y,n3y4V2w0XSK8O92AUsM6,zekUyNFAwIx2rBmVjJAX,f1Vq2wQO5RObCX2pZetw,BTWjsEIjfE4ZVZkHbXCo
count,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
unique,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
top,"{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...",...,"{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco...","{'__datatype__': 'timestamp', 'value': {'_seco..."
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [5]:
# preprocess your test data

# sample iteration over df0
for ssid in df1:
    locationX = df1[ssid]['location']['x']
    locationY = df1[ssid]['location']['y']
    # print(ssid, locationX, locationY) # this prints 404 lines
    # do some other stuff you want to

In [ ]:
# format test data

In [ ]:
# predict the location X and Y using your model

In [6]:
# generate mysubmission
df2 = pd.read_csv('submission.csv')
for i, row in df2.iterrows():
    df2.at[i,'locationX'] = 0
    df2.at[i,'locationY'] = 0

print(df2)

      id                scanid locationX locationY
0      0  KnW7bcCTFKSkaxQM9HwS         0         0
1      1  WVdanpyXB4Le6D7Qlxqm         0         0
2      2  yr3L2JcYm938dXdtCYj0         0         0
3      3  9MGDml7ZSWLVrxkytoR8         0         0
4      4  bFesNkqRjhgvo7bAHQs6         0         0
..   ...                   ...       ...       ...
96    96  dHXIwA3dTzHRZjpxlA3Y         0         0
97    97  n3y4V2w0XSK8O92AUsM6         0         0
98    98  zekUyNFAwIx2rBmVjJAX         0         0
99    99  f1Vq2wQO5RObCX2pZetw         0         0
100  100  BTWjsEIjfE4ZVZkHbXCo         0         0

[101 rows x 4 columns]


In [7]:
# save mysubmission to file mysubmission.csv
df2.to_csv('./mysubmission.csv', index=False)

# Submitting to the CTFSG DataScience Grader
ALT-TAB LABS LLP &copy; 2020-2021 All Rights Reserved

# Using the web interface

Just visit the grader URL provided with the questions and upload using the web interface:
http://ABC.XYZ.ctf.sg:12345/

# Using the python library

In [ ]:
# Download CTFSG Grader Libraries
import urllib.request, os
urllib.request.urlretrieve('https://raw.githubusercontent.com/alttablabs/ctfsg-utils/master/pyctfsglib.py', './pyctfsglib.py')
print('Downloaded pyctfsglib.py:', 'pyctfsglib.py' in os.listdir())

In [ ]:
# Connect to graders
import pyctfsglib as ctfsg
import random

USER_TOKEN = "USER_TOKEN" # You need to fill this up
GRADER_URL = random.choice([ # Check if these are correct GRADER URLS for the specific challenge
    "http://ABC.XYZ.ctf.sg:12345/",
    "http://ABC.XYZ.ctf.sg:23456/"
])

grader = ctfsg.DSGraderClient(GRADER_URL, USER_TOKEN)

In [ ]:
# Submitting a file
grader.submitFile('./mysubmission.csv')